In [64]:
import numpy as np
import pandas as pd

In [65]:
# Code from previous assignments (or based on previous assignments)

class Neuron:
    """Class developed for HW1. Implements a single neuron and a forward pass through that neuron."""
    def __init__(self, weights, activation_function):
        self.weights = np.array(weights)
        self.bias = weights[0]
        self.activation_function = activation_function
        
    def predict(self, inputs):
        # Assumes that inputs are augmented to account for bias
        return self.activation_function(np.dot(self.weights, inputs))

class PerceptronTrainer:
    """Class developed based on submission for HW2. Implements the perceptron training algorithm."""
    def __init__(self, perceptron, learning_rate=0.1):
        self.perceptron = perceptron
        self.learning_rate = learning_rate

    def train(self, training_data, labels):
        num_epochs = 0
        print(f"Epoch {num_epochs}")
        self.report_weights()
        accuracy = self.report_accuracy(training_data, labels)
        
        while accuracy < 1.0:
            num_epochs += 1
            for input_x, label_d in zip(training_data, labels):
                output_y = self.perceptron.predict(input_x)
                self.perceptron.weights += self.learning_rate * (label_d - output_y) * input_x

            print(f"Epoch {num_epochs}")
            self.report_weights()
            accuracy = self.report_accuracy(training_data, labels)

    def report_weights(self):
        print(f'\tWeights: {self.perceptron.weights}')
        return self.perceptron.weights
        
    def report_accuracy(self, training_data, labels):
        predictions = [self.perceptron.predict(inputs) for inputs in training_data]
        accuracy = sum(p == l for p, l in zip(predictions, labels)) / len(labels)
        print(f'\tAccuracy: {accuracy * 100:.2f}%')
        return accuracy

In [66]:
class mnist_loader:
    """Class developed for HW3. Loads MNIST data from the given file path."""
    
    def __init__(self):
        pass
            
    def load_labels(self, filename):
        """MNIST labels are stored as a 1D array of bytes."""
        
        with open(filename, 'rb') as f:
            print(f"Reading data from {filename}...")
            
            magic_number, num_labels = np.frombuffer(f.read(8), dtype='>i4')
            # print(f"Magic number: {magic_number}, Number of labels: {num_labels}")
            
            labels = np.frombuffer(f.read(), dtype=np.uint8)
            print(f"Read {len(labels)} labels.")
    
        return labels
    
    def load_images(self, filename):
        """MNIST images are stored as a 2D array of bytes, with each row representing an image."""
        
        with open(filename, 'rb') as f:
            print(f"Reading data from {filename}...")
            
            magic_number, num_images, num_rows, num_cols = np.frombuffer(f.read(16), dtype='>i4')
            # print(f"Magic number: {magic_number}, Number of images: {num_images}, Image size: {num_rows}x{num_cols}")
            
            images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, num_rows * num_cols)
            print(f"Read {images.shape[0]} images with shape {num_rows}x{num_cols}.")
            
        return images
    
    
def load_training_data():
    loader = mnist_loader()
    train_labels = loader.load_labels('mnist/train-labels.idx1-ubyte')
    train_images = loader.load_images('mnist/train-images.idx3-ubyte')

    return train_labels, train_images

# train_df = load_training_data()
# train_df

In [71]:
class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.weights = np.zeros(shape=self.layers)

    def predict(self, inputs_X):
        # Implement forward pass through the network
        return 0  # Placeholder for predicted class
                
    def gradient_descent(self, inputs_X, labels_d, learning_rate=0.01, accuracy_threshold=0.95):
        num_epochs = 0
        
        print(f"Epoch {num_epochs}")
        accuracy = self.calculate_accuracy(inputs_X, labels_d, report=True)
                
        while accuracy < accuracy_threshold:
            num_epochs += 1
            # Implement forward pass, backpropagation, and weight updates here
            
            print(f"Epoch {num_epochs}")
            accuracy = self.calculate_accuracy(inputs_X, labels_d, report=True)

    def describe_network(self):
        print(f"Network architecture:")
        for layer_idx, layer in enumerate(self.layers):
            print(f"\tLayer {layer_idx + 1}: {layer} neurons")
        
    def calculate_loss(self, inputs_X, labels_d, report=False):
        # Implement loss calculation (e.g., cross-entropy loss)
        return 0.0  # Placeholder for loss value
    
    def calculate_accuracy(self, inputs_X, labels_d, report=False):
        predictions = [self.predict(inputs) for inputs in inputs_X]
        accuracy = sum(p == l for p, l in zip(predictions, labels_d)) / len(labels_d)
        
        if report:
            print(f'\tAccuracy: {accuracy * 100:.2f}%')
        
        return accuracy
        
        

train_labels, train_inputs = load_training_data()    

nn = NeuralNetwork((784, 10))
nn.gradient_descent(train_inputs, train_labels, learning_rate=0.01, accuracy_threshold=0.95)

Reading data from mnist/train-labels.idx1-ubyte...
Read 60000 labels.
Reading data from mnist/train-images.idx3-ubyte...
Read 60000 images with shape 28x28.
Epoch 0
	Accuracy: 9.87%
Epoch 1
	Accuracy: 9.87%
Epoch 2
	Accuracy: 9.87%
Epoch 3
	Accuracy: 9.87%
Epoch 4
	Accuracy: 9.87%
Epoch 5
	Accuracy: 9.87%
Epoch 6
	Accuracy: 9.87%
Epoch 7
	Accuracy: 9.87%
Epoch 8
	Accuracy: 9.87%
Epoch 9
	Accuracy: 9.87%
Epoch 10
	Accuracy: 9.87%
Epoch 11
	Accuracy: 9.87%
Epoch 12
	Accuracy: 9.87%
Epoch 13
	Accuracy: 9.87%
Epoch 14
	Accuracy: 9.87%
Epoch 15
	Accuracy: 9.87%
Epoch 16
	Accuracy: 9.87%
Epoch 17
	Accuracy: 9.87%
Epoch 18
	Accuracy: 9.87%
Epoch 19
	Accuracy: 9.87%
Epoch 20
	Accuracy: 9.87%
Epoch 21
	Accuracy: 9.87%
Epoch 22
	Accuracy: 9.87%
Epoch 23
	Accuracy: 9.87%
Epoch 24
	Accuracy: 9.87%
Epoch 25
	Accuracy: 9.87%
Epoch 26
	Accuracy: 9.87%
Epoch 27
	Accuracy: 9.87%
Epoch 28
	Accuracy: 9.87%
Epoch 29
	Accuracy: 9.87%
Epoch 30
	Accuracy: 9.87%
Epoch 31
	Accuracy: 9.87%
Epoch 32
	Accuracy: 9

KeyboardInterrupt: 